<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [14]:
##### Importing #####
import pandas as pd
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
df.head(5)

##### Cleaning #####

# No null values
df.isnull().sum().sum()

# CustomerID is a unique identifier so drop
df = df.drop(columns=['customerID'])

# Gender
df['gender'].replace({'Male': 1, 'Female': 2}, inplace=True)

# Encoding yes/no's 
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn', 'MultipleLines']
for col in binary_cols:
  df[col].replace({'Yes': 1, 'No': 0, 'No internet service': 0, 'No phone service': 0}, inplace=True)

# Ordinal encoding
df['InternetService'].replace({'Fiber optic': 3, 'DSL': 2, 'No': 1}, inplace=True)
df['Contract'].replace({'Month-to-month': 1, 'One year': 2, 'Two year': 3}, inplace=True)
df['PaymentMethod'].replace({'Credit card (automatic)': 4, 'Bank transfer (automatic)': 3, 'Electronic check': 2, 'Mailed check': 1}, inplace=True)

# Removing rows with spaces and converting to float
df = df[df['TotalCharges'] != ' ']
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

# Normalizing (row-wise operation as compared to StandardScaler)
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
X = transformer.fit_transform(df.drop(columns=['Churn']))

X

array([[4.71529624e-02, 0.00000000e+00, 2.35764812e-02, ...,
        4.71529624e-02, 7.03757963e-01, 7.03757963e-01],
       [5.28913807e-04, 0.00000000e+00, 0.00000000e+00, ...,
        5.28913807e-04, 3.01216413e-02, 9.99382639e-01],
       [8.27287359e-03, 0.00000000e+00, 0.00000000e+00, ...,
        8.27287359e-03, 4.45494243e-01, 8.94711279e-01],
       ...,
       [5.74860378e-03, 0.00000000e+00, 2.87430189e-03, ...,
        5.74860378e-03, 8.50793360e-02, 9.95801890e-01],
       [3.16906801e-03, 3.16906801e-03, 3.16906801e-03, ...,
        3.16906801e-03, 2.35778660e-01, 9.71636251e-01],
       [1.46078464e-04, 0.00000000e+00, 0.00000000e+00, ...,
        4.38235391e-04, 1.54331897e-02, 9.99834045e-01]])

In [15]:
##### Create and fit a baseline Keras MLP model to the data. #####
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Before fitting a little more preprocessing...
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df['Churn'], test_size=0.2, random_state=42)
y_test = np.asarray(y_test)
y_train = np.asarray(y_train)

inputs = X_train.shape[1]

# All numpy arrays
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5625, 19), (1407, 19), (5625,), (1407,))

In [4]:
# Baseline for just guessing majority class
"""
No     0.73463
Yes    0.26537

73% guessing No Churning for every instance
"""

'\nNo     0.73463\nYes    0.26537\n\n73% guessing No Churning for every instance\n'

In [19]:
# Building the baseline model w/ 19 input nodes (# of features) and 2 softmax probability output nodes for the 2 classes 
model = Sequential()
model.add(Dense(19, activation='relu', input_shape=(inputs,)))
model.add(Dense(1, activation='relu'))

# Compiling
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fitting
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, verbose=1)

Epoch 1/20
176/176 [==============================] - 0s 2ms/step - loss: 0.5756 - accuracy: 0.7342 - val_loss: 0.5412 - val_accuracy: 0.7342
Epoch 2/20
176/176 [==============================] - 0s 1ms/step - loss: 0.5340 - accuracy: 0.7444 - val_loss: 0.5267 - val_accuracy: 0.7548
Epoch 3/20
176/176 [==============================] - 0s 1ms/step - loss: 0.5229 - accuracy: 0.7556 - val_loss: 0.5188 - val_accuracy: 0.7605
Epoch 4/20
176/176 [==============================] - 0s 1ms/step - loss: 0.5153 - accuracy: 0.7582 - val_loss: 0.5137 - val_accuracy: 0.7605
Epoch 5/20
176/176 [==============================] - 0s 1ms/step - loss: 0.5092 - accuracy: 0.7600 - val_loss: 0.5063 - val_accuracy: 0.7598
Epoch 6/20
176/176 [==============================] - 0s 2ms/step - loss: 0.5012 - accuracy: 0.7614 - val_loss: 0.4986 - val_accuracy: 0.7619
Epoch 7/20
176/176 [==============================] - 0s 2ms/step - loss: 0.4943 - accuracy: 0.7644 - val_loss: 0.4929 - val_accuracy: 0.7647
Epoch 

In [0]:
"""
Hyperparameter tune (at least) the following parameters:

batch_size
training epochs
optimizer
learning rate (if applicable to optimizer)
momentum (if applicable to optimizer)
activation functions
network weight initialization
dropout regularization
number of neurons in the hidden layer
You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters

Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?
"""

In [0]:
# Hypertuning
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model():

    model = Sequential()
    model.add(Dense(19, activation='relu', input_shape=(inputs,)))
    model.add(Dense(1, activation='relu'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# create model with an sklearn wrapper
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10],
              'epochs': [100, 200, 300, 400, 500, 600, 700]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

In [23]:
# Cross validation
from sklearn.model_selection import cross_val_score
cross_val_score(model, X_train, y_train, cv=3)

array([0.75999999, 0.74346668, 0.72426665])

In [0]:
"""
Batch size: [10, 20, 40, 60, 80, 100] - 10 was the best
Epochs: [100, 200, 300, 400, 500, 600, 700] - 
"""

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?